# Decision Trees

"Like SVMs, **Decision Trees** are versatile Machine Learning algorithms that can perform both classification and regression tasks, and even multioutput tasks.

#### Training and Visualizing a Decision Tree

Let's build and train a simple Decision Tree classifier using Scikit-learn's `DecisionTreeClassifier` class.

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
x = iris.data[:, 2:]
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(x,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

We can easily visualize the Decision Tree. First, we need to use the `export_graphviz()` method to convert it to a file called `iris_tree.dot`. Then we can use the `graphviz` package to convert it to a PDF or PNG with the command:

`$ dot -Tpng iris_tree.dot -o iris_tree.png`

In [3]:
from sklearn.tree import export_graphviz
from IPython.display import Image
import pydotplus

export_graphviz(
    tree_clf,
    out_file='iris_tree.dot',
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

In [4]:
from sklearn.externals.six import StringIO

dot_data = StringIO()

C:\Users\Tim\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [5]:
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())


^
Expected {'graph' | 'digraph'} (at char 0), (line:1, col:1)


AttributeError: 'NoneType' object has no attribute 'create_png'

The prediction making process of decision trees is actually super simple. It simply makes a flowchart-style questionarre. It asks if the pedal width is less than a certain size. If so, it will predict one thing, if not, it may ask a couple more questions before coming to a prediction.

"One of the many qualities of Decision Trees is that they require very little data preparation. In particular, they don't require feature scaling or centering at all."

"Scikit-Learn uses the CART algorithm, which produces only *binary trees*: nonleaf nodes always have two children (i.e., questions only have yes/no answers). However, other algorithms such as ID3 can produce Decision Trees with nodes that have more than two children."

The good news about Decision Trees is that they are considered **white box models**. In that it is very clear why their predictions were made, unlike Random Forests or Neural Networks.

#### Estimating Class Probabilities

"A Decision Tree can also estimate the probability that an instance belongs to a particular class k: first it traverses the tree to find the leaf node for this instance, and then it returns the ratio of training instances of class k in this node. For example, suppose you have found a flower whose petals are 5 cm long and 1.5 cm wide. The corresponding leaf node is the depth-2 left node, so the Decision Tree should output the following probabilities: 0% for Iris-Setosa (0/54), 90.7% for Iris-Versicolor (49/54), and 9.3% for Iris-Virginica (5/54)."

In [6]:
tree_clf.predict_proba([[5, 1.5]])

array([[0.        , 0.90740741, 0.09259259]])

#### The CART Training Algorithm

Sk-learn "uses the **Classification and Regression Tree (CART)** algorithm to train Decision Trees." The idea is, the algorithm "first splits the training set into two subsets using a single feature *k* and a threshold *t_k* (e.g., 'pedal length <= 2.45 cm')."

Then, the algorithm just has to search for the optimal *k* and *t_k* that produces the purest two subsets (weighted by their size). The cost function then is:

J(k, t_k) = (m_left/m) * G_left + (m_right/m) * G_right

where G_left and G_right are the impurities of the left/right subsets respectively, and m_left and m_right are the number of instances in the left/right subsets respectively.

"Once it has successfully split the training set in two, it splits the subsets using the same logic, then the sub-subsets and so on, recursively. It stops recursing once it reaches the maximum depth [...] or if it cannot find a split that will reduce impurity." A few other hyperparameters, min_samples_split, min_samples_leaf, min_weight_fraction_leaf, and max_leaf_nodes, can result in additional early stopping conditions, but they will not be discussed until later.

#### Regularization Hyperparameters

"Decision Trees make very few assumptions about the training data (as opposed to linear models, which obviously assume that the data is linear, for example). If left unconstrained, the tree structure will adapt itself to the training data, fitting it very closely and most likely overfitting it."

Decision Trees are a type of **nomparametric model**, not because it has zero parameters, but because the number of parameters is determined during the training process and not before. Thus, it is more prone to overfitting and must be regularized.

In Scikit-Learn, the `max_depth` hyperparameter that determines the number of 'layers' to the tree has a default value of `None`, so be sure to set a `max_depth` if you want some regularization.

"The `DecisionTreeClassifier` class has a few other parameters that similarly restrict the shape of the Decision Tree:"

- `min_samples_split` is the minimum number of samples a node must have before it can be split
- `min_samles_leaf` is the minimum number of samples a leaf node must have
- `min_weight_fraction_leaf` is the same as `min_samples_leaf` but expressed as a fraction of the total number of weighted instances
- `max_leaf_nodes` is the maximum number of leaf nodes
- `max_features` is the maximum number of features that are evaluated for splitting at each node.

### Regression
Decision Trees are also capable of performing regression tasks. Let's build a regression tree using Scikit-Learn's `DecisionTreeRegressor` class, training it on a noisy quadratic dataset with `max_depth = 2`:

In [2]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np

m = 100
x = 6 * np.random.rand(m, 1) - 3
y = 0.5 * x**2 + x + 2 + np.random.randn(m,1)
tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(x,y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

"The CART algorithm works mostly the same way as earlier, except that instead of trying to split the training set in a way that minimizes impurity, it now tries to split the training set in a way that minimizes the MSE."

Be aware though, Decision Tree Regressors are just as prone to overfittment as their Classifier counterparts. Make sure you regularize.

### Instability

Decision Trees are pretty dope. "They are simple to understand and interpret, easy to use, versatile, and powerful. However, they do have a few limitations. First, as you may have notices, Decision Trees love orthogonal decision boundaries (all splits are perpendicular to an axis), which makes them very sensitive to training set rotation."

Look at how much this Decision Tree is affected by a simple 45 degree rotation of a simply linearly seperable dataset.

![Before](trainingsetRotation.jpg)

"More generally, the main issue with Decision Trees is that they are very sensitive to small variations in training data. For example," take a look at the previous decision boundaries we had of the Iris dataset.

!['Before'](before.jpg)

"If you just remove the widest Iris-Versicolor from the training set... and train a new Decision Tree, you may get [this]:"

!['After'](after.jpg)

"Random Forests can limit this instability by averaging predictions over many trees, as we will see in the next chapter."